In [1]:
from aut import *
from pyspark.sql.functions import col, desc

In [2]:
warcs = "/home/nruest/Projects/au/sample-data/geocities"
results = "/tmp/aut-testing/"

webpages = WebArchive(sc, sqlContext, warcs).webpages()
webgraph = WebArchive(sc, sqlContext, warcs).webgraph()

In [3]:
# Domain frequency.
webpages.groupBy("domain") \
  .count() \
  .sort(col("count")\
  .desc()) \
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "domains")

[2022-06-11T14:54:42.403Z - 00003 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143351-00117-ia400103.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T14:54:42.403Z - 00002 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T14:54:42.403Z - 00004 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T14:54:42.403Z - 00007 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143451-00102-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T14:54:42

22/06/11 10:54:43 WARN PDFParser: J2KImageReader not loaded. JPEG2000 files will not be processed.
See https://pdfbox.apache.org/2.0/dependencies.html#jai-image-io
for optional dependencies.

22/06/11 10:54:43 WARN TesseractOCRParser: Tesseract OCR is installed and will be automatically applied to image files unless
you've excluded the TesseractOCRParser from the default parser.
Tesseract may dramatically slow down content extraction (TIKA-2359).
As of Tika 1.15 (and prior versions), Tesseract is automatically called.
In future versions of Tika, users may need to turn the TesseractOCRParser on via TikaConfig.
22/06/11 10:54:43 WARN SQLite3Parser: org.xerial's sqlite-jdbc is not loaded.
Please provide the jar on your classpath to parse sqlite files.
See tika-parsers/pom.xml for the correct version.


In [4]:
# Domain graph.
webgraph.groupBy("crawl_date", remove_prefix_www(extract_domain("src")).alias("src_domain"), remove_prefix_www(extract_domain("dest")).alias("dest_domain"))\
  .count()\
  .filter((col("dest_domain").isNotNull()) & (col("dest_domain") !=""))\
  .filter((col("src_domain").isNotNull()) & (col("src_domain") !=""))\
  .filter(col("count") > 5)\
  .orderBy(desc("count"))\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "domain-graph")

[2022-06-11T14:59:45.030Z - 00440 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143300-00114-ia400112.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T14:59:45.030Z - 00431 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T14:59:45.030Z - 00432 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143243-00104-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T14:59:45.030Z - 00433 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T14:59:45

In [5]:
# Image graph.
WebArchive(sc, sqlContext, warcs).imagegraph()\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "image-graph")

[2022-06-11T15:03:15.196Z - 00948 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:03:15.196Z - 00950 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:03:15.208Z - 00949 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143351-00117-ia400103.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:03:15.209Z - 00954 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:03:15

In [6]:
# Web graph.
webgraph.write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "web-graph")

[2022-06-11T15:05:52.896Z - 00964 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:05:52.896Z - 00960 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:05:52.897Z - 00957 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143243-00104-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:05:52.896Z - 00961 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143340-00105-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:05:52

In [7]:
# Web pages.
webpages.write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "webpages")

[2022-06-11T15:08:59.968Z - 00970 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:08:59.968Z - 00974 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:08:59.968Z - 00975 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143300-00114-ia400112.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:08:59.970Z - 00973 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143451-00102-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:09:00

[2022-06-11T15:09:00.568Z - 00971 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143340-00105-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:09:00.722Z - 00966 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


In [8]:
# Binary information.
WebArchive(sc, sqlContext, warcs).audio()\
  .select("crawl_date", "url", "filename", "extension", "mime_type_web_server", "mime_type_tika", "md5", "sha1")\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "audio")

[2022-06-11T15:13:55.268Z - 00985 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143300-00114-ia400112.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:13:55.269Z - 00977 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143243-00104-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:13:55.268Z - 00981 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143340-00105-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:13:55.270Z - 00978 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:13:55

In [9]:
WebArchive(sc, sqlContext, warcs).images()\
  .select("crawl_date", "url", "filename", "extension", "mime_type_web_server", "mime_type_tika", "width", "height", "md5", "sha1")\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "images")

[2022-06-11T15:17:08.303Z - 00988 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:17:08.303Z - 00989 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143351-00117-ia400103.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:17:08.303Z - 00990 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:17:08.303Z - 00994 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:17:08

In [10]:
WebArchive(sc, sqlContext, warcs).pdfs()\
  .select("crawl_date", "url", "filename", "extension", "mime_type_web_server", "mime_type_tika", "md5", "sha1")\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "pdfs")

[2022-06-11T15:25:52.543Z - 00999 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143351-00117-ia400103.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:25:52.543Z - 00998 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:25:52.543Z - 01002 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143841-00136-ia400104.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:25:52.575Z - 01003 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143451-00102-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:25:52

[2022-06-11T15:25:53.014Z - 01005 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143300-00114-ia400112.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


In [11]:
WebArchive(sc, sqlContext, warcs).presentation_program()\
  .select("crawl_date", "url", "filename", "extension", "mime_type_web_server", "mime_type_tika", "md5", "sha1")\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "presentation_program")

[2022-06-11T15:28:52.256Z - 01010 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:28:52.258Z - 01015 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143300-00114-ia400112.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:28:52.261Z - 01007 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143243-00104-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:28:52.261Z - 01012 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143841-00136-ia400104.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:28:52

[2022-06-11T15:28:52.987Z - 01006 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


In [12]:
WebArchive(sc, sqlContext, warcs).spreadsheets()\
  .select("crawl_date", "url", "filename", "extension", "mime_type_web_server", "mime_type_tika", "md5", "sha1")\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "spreadsheets")

[2022-06-11T15:32:18.570Z - 01020 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:32:18.570Z - 01022 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143841-00136-ia400104.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:32:18.571Z - 01018 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:32:18.571Z - 01024 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:32:18

In [13]:
WebArchive(sc, sqlContext, warcs).video()\
  .select("crawl_date", "url", "filename", "extension", "mime_type_web_server", "mime_type_tika", "md5", "sha1")\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "videos")

[2022-06-11T15:36:22.717Z - 01029 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143351-00117-ia400103.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:36:22.717Z - 01032 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143841-00136-ia400104.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:36:22.717Z - 01030 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:36:22.733Z - 01028 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:36:22

[2022-06-11T15:36:23.430Z - 01026 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])


In [14]:
WebArchive(sc, sqlContext, warcs).word_processor()\
  .select("crawl_date", "url", "filename", "extension", "mime_type_web_server", "mime_type_tika", "md5", "sha1")\
  .write\
  .option("timestampFormat", "yyyy/MM/dd HH:mm:ss ZZ")\
  .format("csv")\
  .option("escape", "\"")\
  .option("encoding", "utf-8")\
  .save(results + "word_processor")

[2022-06-11T15:39:37.843Z - 01038 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:39:37.859Z - 01044 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:39:37.859Z - 01037 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143243-00104-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:39:37.859Z - 01043 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143451-00102-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-06-11T15:39:37

[2022-06-11T15:39:38.429Z - 01036 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143512-00103-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
